In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_3b_NonPath_SP2_EsCo_PSL_1_2_3_4_574.csv') 

# split data
X=data.iloc[:,1:51]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A     Xc1.R     Xc1.N  ...  Xc2.lambda28  Xc2.lambda29  Xc2.lambda30
0     2.566136  2.332851  0.233285  ...      0.027442      0.028477      0.026934
1    17.798460  6.396322  6.396322  ...      0.022623      0.025014      0.024275
2     5.387067  2.835298  1.984709  ...      0.025133      0.025211      0.022471
3    13.095425  1.636928  4.583399  ...      0.022593      0.021866      0.023481
4     9.634552  5.831440  3.549572  ...      0.024823      0.025635      0.025564
..         ...       ...       ...  ...           ...           ...           ...
569  16.638587  8.839250  8.059316  ...      0.023628      0.026980      0.025744
570  13.692819  4.471124  6.147796  ...      0.024647      0.024883      0.022908
571   4.115225  3.566528  2.194787  ...      0.026345      0.021196      0.024668
572  12.470586  6.495097  7.014705  ...      0.023367      0.024836      0.024051
573   4.793531  1.597844  2.929380  ...      0.025786      0.024418      0.024007

[574 rows x 50 

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(50, 4 - 1) = 3 components.
  ChangedBehaviorWarni

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 151,716
Trainable params: 151,716
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
30/30 [==============================] - 33s 109ms/step - loss: 0.6719 - accuracy: 0.4503 - val_loss: 0.5371 - val_accuracy: 0.4449
Epoch 2/1000
30/30 [==============================] - 1s 20ms/step - loss: 0.4953 - accuracy: 0.4519 - val_loss: 0.4532 - val_accuracy: 0.5678
Epoch 3/1000
30/30 [==============================] - 1s 20ms/step - loss: 0.4597 - accuracy: 0.5176 - val_loss: 0.4111 - val_accuracy: 0.6102
Epoch 4/1000
30/30 [==============================] - 1s 20ms/step - loss: 0.4174 - accuracy: 0.6183 - val_loss: 0.3598 - val_accuracy: 0.6525
Epoch 5/1000
30/30 [==============================] - 1s 21ms/step - loss: 0.3807 - accuracy: 0.6399 - val_loss: 0.3410 - val_accuracy: 0.7119
Epoch 6/1000
30/30 [==============================] - 1s 19ms/step - loss: 0.3666 - accuracy: 0.6484 - val_loss: 0.3193 - val_accuracy: 0.7203
Epoch 7/1000
30/30 [==============================] - 1s 20ms/step - loss: 0.3435 - accuracy: 0.7034 - val_loss: 0.2899 - val_accuracy: 0.75

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.8914893617021277
test-acc = 0.8432203389830508
train-CFR =               precision    recall  f1-score   support

           0       0.85      0.87      0.86       230
           1       0.88      0.89      0.89       241
           2       0.91      0.94      0.92       227
           3       0.93      0.87      0.90       242

    accuracy                           0.89       940
   macro avg       0.89      0.89      0.89       940
weighted avg       0.89      0.89      0.89       940

test-CFR =               precision    recall  f1-score   support

           0       0.85      0.70      0.77        64
           1       0.72      0.83      0.77        53
           2       0.92      0.97      0.94        67
           3       0.88      0.87      0.87        52

    accuracy                           0.84       236
   macro avg       0.84      0.84      0.84       236
weighted avg       0.85      0.84      0.84       236

